In [64]:
import pandas as pd
import os
import re

In [65]:
# Establecimientos Fundadoras
#   Voy a tratar de hacer un input similar al que utiliza el modelo ("coordenadas_campos.xlsx"),tiene las columnas ["Zona","Campo","Lat", ""Long].
#       Asi de esa manera puedo correr el modelo de geolocalizacion_establecimientos.py, con el archivo generado.
#   Voy a crear una carpeta que se llame input_establecimientos, que va a ser consumido por dicho modelo.

In [66]:
# Paths
name_coordenadas_campos = '../Input/coordenadas_campos.xlsx'
name_establecimientos_fundadora = '../input_bauti/ultima_campaña/Establecimientos Fundadora.xlsx'

# Maestro de establecimientos.
name_maestro_establecimientos = '../Input_fijo/maestro_establecimientos.xlsx'

# Listado de lotes, de aca obtenemos las distintas areas para todos los establecimientos, ademas del Good Supplier.
name_listado_lotes = '../input_bauti/ultima_campaña/Listado de lotes.xlsx'
# Output.
path_output = '../input_bauti/coordenadas_campos/'
name_output = 'establecimientos_coordenadas.xlsx'

In [67]:
# Load
df_maestro_establecimiento = pd.read_excel(name_maestro_establecimientos)
df_coordenadas_campos = pd.read_excel(name_coordenadas_campos)
df_fundadora = pd.read_excel(name_establecimientos_fundadora)
df_lotes  = pd.read_excel(name_listado_lotes)

# Listado Lotes

Aca vamos a agrupar por los distintas subzonas que pueda tener un establecmiento, asi tendremos unificado las subzonas para cada establecmiento.

In [68]:
# Veo datos de los lotes.
df_lotes.sample(1)

,Zona,Subzona,Lote,Field Master Data,Localidad,CAMPO,Km,TMIN,TMAX,Area,Good Supplier (SAP)
265,S,S,XS4733,La Esperanza,Chacabuco,La Esperanza,73.0,01:00:00,02:00:00,Comercial,878875


In [69]:
# Me quedo con los campos que me interesan.
df_lotes = df_lotes[['Zona', 'Field Master Data', 'Area', 'Good Supplier (SAP)']]
df_lotes.columns = ['zona', 'campo' ,'area', 'good_supplier']

# Elimino subzonas que pudieran llegar a aparecer.
df_lotes['zona'] = df_lotes['zona'].str.replace(r'\d*','', regex=True)

# Ignoro zonas con nombre "AT".
mask_no_AT = df_lotes['zona']!='AT'
df_lotes = df_lotes.loc[mask_no_AT, :]

df_lotes['campo'] = df_lotes['campo'].str.lower()
df_lotes['area'] = df_lotes['area'].str.lower()
df_lotes.sample(1)

,zona,campo,area,good_supplier
538,B,fran luis,fundadora,879002


In [70]:
# Agrupo por cada campo para obtener los distitnos establecimientos.
df_lotes = df_lotes.groupby(['zona', 'campo', 'area']).value_counts().reset_index().drop(columns=0)
df_lotes = df_lotes[['zona', 'campo', 'area', 'good_supplier']]
# Este DF se usara para obtener las distintas areas y el good supplier de cada establecmiento.
print('Shape Listado Lotes - %s Filas / %s Columnas' %(df_lotes.shape[0], df_lotes.shape[1]))
df_lotes.sample(3)

Shape Listado Lotes - 165 Filas / 4 Columnas


,zona,campo,area,good_supplier
7,C,mistol ancho,comercial,932367
148,X,el pilar 1,comercial,892157
121,T,ercole,fundadora,879001


# Establecimientos fundadora.

Vamos a preparar los datos de establecimiento fundadora. Y ver si estos datos coinciden con el maestro de establecimientos.

In [71]:
# Dropea columnas que posean nullos ('Unnamed')
df_fundadora.dropna(axis=1, how='all', inplace=True)
# Dropeo fillas que contienen algun nulo.
df_fundadora.dropna(axis=0, how='any', inplace=True)
df_fundadora.rename(columns={'Field Master Data' : 'Campo'}, inplace=True)

# Selecciono los campo utiles.
df_fundadora = df_fundadora[['Zona', 'Campo', 'Good Supplier', 'Lat', 'Long']]
df_fundadora.columns = ['zona', 'campo', 'good_supplier', 'lat', 'long']

# Convierto los establecimientos a minuscula ya que asi se encuentran en el maestro de materiales.
df_fundadora['campo'] = df_fundadora['campo'].str.lower()

# Hay presencia de registros duplicados en fundadora.
df_fundadora.drop_duplicates(inplace=True)
df_fundadora.reset_index(inplace=True, drop=True)

In [72]:
# Convierto las Lat y Long a strings para poder aplicar Regex.
df_fundadora['lat'] = df_fundadora['lat'].astype(str)
df_fundadora['long'] = df_fundadora['long'].astype(str)

# La longitud y Latitud se encuentran separadas por ',' en vez de por '.', unifico.
patron = '(-\d{2}),?\.?(\d*)'
patron_re = re.compile(patron)

# Aplico Regex.
df_fundadora['lat'] = (df_fundadora['lat'].apply(lambda x: patron_re.search(x).group(1) + '.' + patron_re.search(x).group(2))).astype('float64')
df_fundadora['long'] = (df_fundadora['long'].apply(lambda x: patron_re.search(x).group(1) + '.' + patron_re.search(x).group(2))).astype('float64')

In [73]:
print('Shape Fundadora - %s Filas / %s Columnas' %(df_fundadora.shape[0], df_fundadora.shape[1]))
df_fundadora.sample(3)

Shape Fundadora - 29 Filas / 5 Columnas


,zona,campo,good_supplier,lat,long
4,E,la sarita 2,879007,-33.898870,-60.178470
17,T,el pelon,932918,-34.122569,-60.671617
16,T,don domingo,932916,-34.290131,-60.318183


In [74]:
mask_establecimiento_no_incluido = df_fundadora['campo'].isin(df_maestro_establecimiento['establecimiento']) == False
df_fundadora_no_incluidos = df_fundadora.loc[mask_establecimiento_no_incluido,:]
# Todos los establecimientos fundadora estan incluidos en el maestro de establecimientos.
df_fundadora_no_incluidos

,zona,campo,good_supplier,lat,long


In [75]:
# Ahora voy a ver si las coordenadas coinciden entre el maestro de establecimientos y fundadora.
# Defino una fucion que me checkee si la fila ingresada es igual a la del maestro fundadora.
def comprobacion_coord (row):
    campo_fundadora = row['campo']
    lat_campo_fundadora = str(row['lat'])
    long_campo_fundadora = str(row['long'])
    
    maestro_establecimiento_row = df_maestro_establecimiento[df_maestro_establecimiento['establecimiento'] == campo_fundadora]
    lat_maestro_establecimiento = str(maestro_establecimiento_row['latitud'].values[0])
    long_maestro_establecimiento = str(maestro_establecimiento_row['longitud'].values[0])
    
    condicion_1 = (lat_campo_fundadora == lat_maestro_establecimiento)
    condicion_2 = (long_campo_fundadora == long_maestro_establecimiento)
    
    if (condicion_1) & (condicion_2):
        return False
    else:
        return True

In [76]:
# Creo mascara para ver que establecimientos tienen mal ingresada las coordenadas.
mask = df_fundadora.apply(comprobacion_coord, axis=1)
# Aplico mascara y genero nuevo df.
df_establecimientos_erroneos = df_fundadora.loc[mask,['campo', 'lat', 'long']]

df_establecimientos_erroneos.rename(columns={'campo':'establecimiento', 'lat':'latitud','long':'longitud'}, inplace=True)

df_establecimientos_erroneos = df_establecimientos_erroneos.merge(
    df_maestro_establecimiento,
    how='inner',  
    on='establecimiento', 
    suffixes=('_fundadora', '_maestro')
    )
df_establecimientos_erroneos

,establecimiento,latitud_fundadora,longitud_fundadora,latitud_maestro,longitud_maestro,Duplicado
0,la sarita 2,-33.898870,-60.178470,-33.397590,-63.574510,0
1,doña clementina,-33.982989,-59.962711,-33.969508,-59.953436,0


In [77]:
establecimientos_erroneos = df_establecimientos_erroneos['establecimiento'].values.tolist()

for establecimiento in establecimientos_erroneos:
    mask_establecimiento_erroneo = df_fundadora['campo'] == establecimiento
    
    valor_lat_correcto = float(df_establecimientos_erroneos.loc[df_establecimientos_erroneos['establecimiento'] == establecimiento, 'latitud_maestro'])
    valor_long_correcto = float(df_establecimientos_erroneos.loc[df_establecimientos_erroneos['establecimiento'] == establecimiento, 'longitud_maestro'])
    
    df_fundadora.loc[mask_establecimiento_erroneo, 'lat'] = valor_lat_correcto
    df_fundadora.loc[mask_establecimiento_erroneo, 'long'] = valor_long_correcto


# Coordenadas campos

Vamos a acomodar los campos, y vamos a agregarle el good supplier

In [78]:
df_coordenadas_campos.columns = ['zona', 'campo', 'lat', 'long']
df_coordenadas_campos['campo'] = df_coordenadas_campos['campo'].str.lower()
print('Shape Coordenadas Campos - %s Filas / %s Columnas' %(df_coordenadas_campos.shape[0], df_coordenadas_campos.shape[1]))
df_coordenadas_campos.sample(3)

Shape Coordenadas Campos - 142 Filas / 4 Columnas


,zona,campo,lat,long
119,V2,santa maría,-33.524451,-61.897438
31,F3,la tijereta,-32.846119,-59.830150
137,Z2,don pancho,-38.136807,-61.441387


In [79]:
# Vamos a agregar el good suplier, mediante un merge.

# Previo a esto guardo las subzonas, ya que voy a eliminar las subzonas.
save_subzonas = df_coordenadas_campos['zona']

# Elimino las subzonas y me quedo unicamente con las zonas, para el merge.
df_coordenadas_campos['zona'] = df_coordenadas_campos['zona'].str.replace(r'\d*','', regex=True)

# Ejecuto merge.
df_coordenadas_campos = df_coordenadas_campos.merge(df_lotes, how='left', on=['zona', 'campo'])

# Selecciono columnas 
df_coordenadas_campos = df_coordenadas_campos[['zona', 'campo', 'lat', 'long', 'good_supplier']]
df_coordenadas_campos['zona'] = save_subzonas

print('Shape Coordenadas Campos - %s Filas / %s Columnas' %(df_coordenadas_campos.shape[0], df_coordenadas_campos.shape[1]))
df_coordenadas_campos.sample(3)

Shape Coordenadas Campos - 142 Filas / 5 Columnas


,zona,campo,lat,long,good_supplier
26,F5,la karina,-32.625049,-65.148419,932426.0
125,X1,la federala 2,-34.097559,-61.035053,922697.0
138,Z2,la emilia,-37.972461,-61.589788,NaN


# Concatenacion de datos.

In [80]:
# Concateno 
df_coordenadas_final = pd.concat([df_coordenadas_campos, df_fundadora], ignore_index=True)

# Elimino duplicados.
df_coordenadas_final.drop_duplicates(inplace=True)

# Reseteo indice.
df_coordenadas_final.reset_index(drop=True, inplace=True)

print('Shape Coordenadas Finales - %s Filas / %s Columnas' %(df_coordenadas_final.shape[0], df_coordenadas_final.shape[1]))
df_coordenadas_final.sample(3)


Shape Coordenadas Finales - 167 Filas / 5 Columnas


,zona,campo,lat,long,good_supplier
37,F5,preve,-32.534797,-65.162554,932432.0
25,F2,la genoveva,-33.634366,-60.067745,932425.0
21,F3,la cabaña 2,-32.761267,-59.839881,922668.0


In [81]:
# Obtengo el area de cada establecimiento.
# Guardo las subzonas.
save_subzonas = df_coordenadas_final['zona']

# Elimino las subzonas y me quedo unicamente con las zonas, para el merge.
df_coordenadas_final['zona'] = df_coordenadas_final['zona'].str.replace(r'\d*','', regex=True)

# Ejecuto merge.
df_coordenadas_final = df_coordenadas_final.merge(df_lotes, how='left', on=['zona', 'campo', 'good_supplier'])

df_coordenadas_final['zona'] = save_subzonas

print('Shape Coordenadas Finales - %s Filas / %s Columnas' %(df_coordenadas_final.shape[0], df_coordenadas_final.shape[1]))
df_coordenadas_final.sample(3)

Shape Coordenadas Finales - 167 Filas / 6 Columnas


,zona,campo,lat,long,good_supplier,area
15,F3,el milagro,-35.845565,-60.492036,932370.0,comercial
114,V1,la adelita,-33.805993,-61.452571,878948.0,comercial
96,S,los angeles,-34.442803,-60.204789,878874.0,comercial


In [82]:
'''
    Puede ser que algunos establecimientos hayan quedado con 'good_supplier' y 'area' en Nan.
    Esto se debe a que no estaban presentes en el listado de lotes, y no son establecimientos 
        fundadora.
    good_supplier quedara en Nan, pero area la imputaremos como comercial.
'''
mask_good_supplier_nan = df_coordenadas_final['good_supplier'].isna()
mask_area_nan = df_coordenadas_final['area'].isna()

df_coordenadas_final[mask_good_supplier_nan | mask_area_nan]

,zona,campo,lat,long,good_supplier,area
134,Z2,la emilia,-37.972461,-61.589788,NaN,NaN


In [83]:
df_coordenadas_final['area'] = df_coordenadas_final['area'].fillna('comercial')

# Acomodo columnas.
df_coordenadas_final = df_coordenadas_final[['zona', 'campo', 'good_supplier', 'area', 'lat', 'long']]
# Acomodo nombre de columna.
df_coordenadas_final.columns = ['Zona', 'Campo', 'Good Supplier', 'Area', 'Lat' ,'Long']

In [84]:
# Creo una nueva carpeta para este archivo historico y guardo el archivo.
if not os.path.exists(path_output):
    os.makedirs(path_output)

df_coordenadas_final.to_excel(path_output+name_output, index=False)